In [ ]:
#Google Colab Link: https://colab.research.google.com/drive/18Q7wLrFuXruZ2OulBggtHUlm1dZH8Sw3?usp=sharing

In [ ]:
!pip install opendatasets

!pip install pandas

In [ ]:

import opendatasets as od
import pandas

od.download(
    "https://www.kaggle.com/datasets/l3llff/flowers")

In [ ]:
!tar -xvzf /content/lfwpeople/lfw-funneled.tgz

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Conv2D, UpSampling2D, BatchNormalization, Activation, Dropout
from keras.models import Sequential, load_model
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb
from skimage.transform import resize
from skimage.io import imsave
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image

In [ ]:
path = '/content/flowers'

train_datagen = ImageDataGenerator(rescale=1. / 255)

train = train_datagen.flow_from_directory(path, target_size=(128, 128), batch_size=5000, class_mode=None)


In [ ]:
X =[]
Y =[]
i = 0
for img in train[0]:
    try:
        lab = rgb2lab(img)
        X.append(lab[:,:,0])
        Y.append(lab[:,:,1:] / 128 )
        i += 1
    except:
      print('error')
X = np.array(X)
Y = np.array(Y)
X = X.reshape(X.shape+(1,))
print(X.shape)
print(Y.shape)

In [ ]:
del train

In [ ]:
import tensorflow as tf

model = tf.keras.Sequential()

model.add(tf.keras.layers.Conv2D(64, kernel_size=3, strides=1, padding='same', use_bias=True, input_shape=(None, None, 1)))
model.add(tf.keras.layers.ReLU())

model.add(tf.keras.layers.Conv2D(64, kernel_size=3, strides=2, padding='same', use_bias=True))
model.add(tf.keras.layers.ReLU())

model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.Conv2D(128, kernel_size=3, strides=1, padding='same', use_bias=True))
model.add(tf.keras.layers.ReLU())

model.add(tf.keras.layers.Conv2D(128, kernel_size=3, strides=2, padding='same', use_bias=True))
model.add(tf.keras.layers.ReLU())

model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.Conv2D(256, kernel_size=3, strides=1, padding='same', use_bias=True))
model.add(tf.keras.layers.ReLU())

model.add(tf.keras.layers.Conv2D(256, kernel_size=3, strides=1, padding='same', use_bias=True))
model.add(tf.keras.layers.ReLU())

model.add(tf.keras.layers.Conv2D(256, kernel_size=3, strides=2, padding='same', use_bias=True))
model.add(tf.keras.layers.ReLU())

model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.Conv2D(512, kernel_size=3, strides=1, padding='same', use_bias=True))
model.add(tf.keras.layers.ReLU())

model.add(tf.keras.layers.Conv2D(512, kernel_size=3, strides=1, padding='same', use_bias=True))
model.add(tf.keras.layers.ReLU())

model.add(tf.keras.layers.Conv2D(512, kernel_size=3, strides=1, padding='same', use_bias=True))
model.add(tf.keras.layers.ReLU())

model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.Conv2D(512, kernel_size=3, strides=1, padding='same', dilation_rate=2, use_bias=True))
model.add(tf.keras.layers.ReLU())

model.add(tf.keras.layers.Conv2D(512, kernel_size=3, strides=1, padding='same', dilation_rate=2, use_bias=True))
model.add(tf.keras.layers.ReLU())

model.add(tf.keras.layers.Conv2D(512, kernel_size=3, strides=1, padding='same', dilation_rate=2, use_bias=True))
model.add(tf.keras.layers.ReLU())

model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.Conv2D(512, kernel_size=3, strides=1, padding='same', dilation_rate=2, use_bias=True))
model.add(tf.keras.layers.ReLU())

model.add(tf.keras.layers.Conv2D(512, kernel_size=3, strides=1, padding='same', dilation_rate=2, use_bias=True))
model.add(tf.keras.layers.ReLU())

model.add(tf.keras.layers.Conv2D(512, kernel_size=3, strides=1, padding='same', dilation_rate=2, use_bias=True))
model.add(tf.keras.layers.ReLU())

model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.Conv2DTranspose(256, kernel_size=4, strides=2, padding='same', use_bias=True))
model.add(tf.keras.layers.ReLU())

model.add(tf.keras.layers.Conv2D(256, kernel_size=3, strides=1, padding='same', use_bias=True))
model.add(tf.keras.layers.ReLU())

model.add(tf.keras.layers.Conv2D(256, kernel_size=3, strides=1, padding='same', use_bias=True))
model.add(tf.keras.layers.ReLU())

model.add(tf.keras.layers.Conv2D(313, kernel_size=1, strides=1, padding='valid', use_bias=True))

softmax_layer = tf.keras.layers.Softmax(axis=3)
model.add(softmax_layer)

output_layer = tf.keras.layers.Conv2D(2, kernel_size=1, strides=1, padding='valid', use_bias=False)
model.add(output_layer)

upsample_layer = tf.keras.layers.UpSampling2D(size=(4, 4), interpolation='bilinear')
model.add(upsample_layer)




In [ ]:
model.summary()

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=0.003)
model.compile( loss='mse' , metrics=['accuracy'])

In [ ]:
history = model.fit(X,Y,validation_split=0.1, epochs=30, batch_size=16)

print(history.history.keys())

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
model.save('/content/test.h5')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
model = load_model('flowers.h5')

In [ ]:
def predict(filename):
  img1_color=[]
  img1=img_to_array(load_img(filename))
  img1 = resize(img1 ,(128,128))
  img1_color.append(img1)
  img1_color = np.array(img1_color, dtype=float)
  img1_color = rgb2lab(1.0/255*img1_color)[:,:,:,0]
  img1_color = img1_color.reshape(img1_color.shape+(1,))
  output1 = model.predict(img1_color)
  output1 = output1*128
  result = np.zeros((128, 128, 3))
  result[:,:,0] = img1_color[0][:,:,0]
  result[:,:,1:] = output1[0]
  img= load_img(filename)
  img2 = lab2rgb(result)
  Titles =["Original", "Prediction"]
  images =[img, img2]
  count = 2

  for i in range(count):
      plt.subplot(1, 2, i + 1)
      plt.title(Titles[i])
      plt.imshow(images[i])

  plt.show()


In [ ]:
predict('/content/testyyy.jpg')